# Preprocessing - classificatie van nieuwsartikelen

In deze notebook gaan we nieuwsartikelen classificeren. Om dit met pytorch zelf te doen heb je wat meer manueel preprocessing werk aangezien de torchtext momenteel niet ondersteund wordt in combinatie met de laatste versie met pytorch.
Om deze reden gaan we hier verder werken met Keras met een pytorch backend.

In [17]:
# Import necessary libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GlobalAveragePooling1D
from keras_preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import opendatasets as od

## Data inladen - Nieuwsberichten

We gebruiken de AG_NEWS dataset die gedownload kan worden van kaggle met deze link: https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

In [21]:
od.download("https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset")

# Load the dataset
def read_csv(filename):
    df = pd.read_csv(filename)
    df.columns = ["label", "title", "description"]
    df["text"] = df['title'] + ' ' + df['description']
    df['label'] = df['label'] - 1
    return df

df_train = read_csv('./ag-news-classification-dataset/train.csv')
display(df_train.head())

df_test = read_csv('./ag-news-classification-dataset/test.csv')

Skipping, found downloaded files in "./ag-news-classification-dataset" (use force=True to force download)


,label,title,description,text
0,2,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,2,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,2,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,2,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


## Preprocessing - tokenizer

## Neuraal netwerk met embedding